In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.datasets import mnist
from sklearn.metrics import mean_squared_error

In [7]:
df = pd.read_csv('../Data Preprocessing/Finalized Out.csv')
df.tail(10)
df.columns = ['Series', 'Day', 'Day_of_Week', 'Time', 'Hour_of_Day', 'IsEJeep1',
       'IsEJeep2', 'Altitude', 'Humidity', 'Temperature',
       'Latitude', 'Longitude', 'Lat Diff', 'Long Diff', 'Distance',
       'Cum Distance', 'Abs Distance', 'Percent Distance', 'Encoded Station',
       'IsStation', 'IsCharging', 'Encoded Prev Station', 'Prev Station Lat',
       'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance',
       'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex',
       'Previous Time to Loyola House of Studies',
       'Previous Time to Grade School', 'Previous Time to Gate 2.5',
       'Previous Time to Leong Hall', 'Encoded Next Station',
       'Next Station Lat', 'Next Station Long', 'Next Time to Xavier Hall',
       'Next Time to Fine Arts Annex', 'Next Time to Loyola House of Studies',
       'Next Time to Grade School', 'Next Time to Gate 2.5',
       'Next Time to Leong Hall']
df.drop(columns=['Series'], inplace=True)

df = df.fillna(-1)
df

,Day,Day_of_Week,Time,Hour_of_Day,IsEJeep1,IsEJeep2,IsEJeep3,Altitude,Humidity,Temperature,...,Previous Time to Leong Hall,Encoded Next Station,Next Station Lat,Next Station Long,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
0,17,4,21600,6,1,0,0,63.7,79.199997,29.299999,...,-1.0,1.0,14.634715,121.076249,22682.0,24486.0,22905.0,21987.0,24948.0,22353.0
1,17,4,21601,6,1,0,0,63.5,79.199997,29.299999,...,-1.0,1.0,14.634715,121.076249,22682.0,24486.0,22905.0,21987.0,24948.0,22353.0
2,17,4,21602,6,1,0,0,63.3,79.199997,29.299999,...,-1.0,1.0,14.634715,121.076249,22682.0,24486.0,22905.0,21987.0,24948.0,22353.0
3,17,4,21603,6,1,0,0,63.0,79.199997,29.299999,...,-1.0,1.0,14.634715,121.076249,22682.0,24486.0,22905.0,21987.0,24948.0,22353.0
4,17,4,21604,6,1,0,0,62.7,79.199997,29.299999,...,-1.0,1.0,14.634715,121.076249,22682.0,24486.0,22905.0,21987.0,24948.0,22353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512132,12,6,64796,17,0,1,0,75.0,80.400002,30.799999,...,35202.0,5.0,14.640187,121.078520,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
512133,12,6,64797,17,0,1,0,75.0,80.400002,30.799999,...,35202.0,5.0,14.640187,121.078520,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
512134,12,6,64798,17,0,1,0,75.0,80.400002,30.799999,...,35202.0,5.0,14.640187,121.078520,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
512135,12,6,64799,17,0,1,0,75.0,80.400002,30.799999,...,35202.0,5.0,14.640187,121.078520,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [8]:
# split data into train-test set
train_df = df[df['Day'] < 16]
test_df = df[df['Day'] >= 16]

train_df = df[df.columns[:-6]]
test_df = df[df.columns[:-6]]

In [10]:
# Define the autoencoder model as a function
def create_autoencoder(encoding_dim=32):
    input_dim = train_df.shape[1]
    input_img = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_img)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

# Wrap the Keras model with scikit-learn's KerasRegressor
autoencoder = KerasRegressor(build_fn=create_autoencoder, verbose=0, encoding_dim=16)

# Define hyperparameters to tune
param_grid = {
    'encoding_dim': [16, 32, 64],
    'epochs': [10, 20],
    'batch_size': [128, 256]
}

# Extract NumPy arrays from DataFrames
x_train = train_df.values
x_test = test_df.values

# Create GridSearchCV
grid = GridSearchCV(estimator=autoencoder, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(x_train, x_train)

# Print the best parameters and corresponding score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error i

Best: -516856695732.333496 using {'batch_size': 128, 'encoding_dim': 32, 'epochs': 20}


NameError: name 'mean_squared_error' is not defined

In [ ]:
# Evaluate the best model on the test set
best_autoencoder = grid_result.best_estimator_
encoded_train = best_autoencoder.predict(x_train)
encoded_test = best_autoencoder.predict(x_test)

# Calculate reconstruction error (MSE) on the test set
mse_test = mean_squared_error(x_test, encoded_test)
print("Test set MSE: ", mse_test)